In [ ]:
import math
import sys

import IPython
import IPython.display as ipd
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

%reload_ext autoreload
%autoreload 2

%matplotlib inline
#%matplotlib notebook

from matplotlib import rcParams
rcParams["figure.max_open_warning"] = False

# 1. Frequency calibration

In [ ]:
exp_name = '2020_12_11_calibration'
fname = f'results/{exp_name}_real.pkl'

try:
    df_total = pd.read_pickle(fname)
    print('read', fname)
except:
    print(f'could not read {fname}, run wall_analysis to parse.')
df_total.tail()

In [ ]:
from pandas_utils import filter_by_dict
from wall_detector import WallDetector
from frequency_analysis import plot_spectrogram
import progressbar

from wall_detector import kwargs_standard
params_dict = {
    'sweep': kwargs_standard,
    'sweep_buzzer': {
        'delta': 50,
        'offset': 0,
        'slope': 5000/128,
        'min_freq': 0,
        'max_freq': 5000,
        'max_time': 135,
    }
}

dict_chosen = {
    'motors': 0,
    'snr': 1,
}
n_spurious = 1 #None

df_chosen = filter_by_dict(df_total, dict_chosen)

for source, df_source in df_chosen.groupby('source'):
    params = params_dict.get(source)
    with progressbar.ProgressBar(max_value=max(df_source.index)) as p:
        for j, row in df_source.iterrows():
            wall_detector = WallDetector(params=params, mic_type='audio_deck')

            spec_masked, freqs_masked = wall_detector.fill_from_row(row, verbose=False, mask=True)
            wall_detector.remove_spurious_freqs(n_spurious, verbose=False)

            psd, psd_std, freqs = wall_detector.get_frequency_slice_with_std()

            fig, axs  = plt.subplots(1, 2)
            fig.set_size_inches(10, 5)
            fig.suptitle(f'experiment="{row.appendix}", snr={row.snr}')

            plot_spectrogram(x=row.seconds, y=freqs_masked, matrix=spec_masked, ax=axs[0])

            for i in range(psd.shape[0]):
                axs[1].errorbar(freqs, psd[i], psd_std[i], label=f'mic{i}')
            axs[1].set_yscale('log')
            plt.legend(loc='upper left')
            p.update(j)
            axs[1].set_ylim(1e-2, 50)

In [ ]:
dict_chosen = {
    'motors': 0,
    'snr': 1,
}
#chosen experiments for calibration
#chosen_experiments = ['_HALL', '_HALL2', '_HALL3']
chosen_experiments = ['_HALL', '_HALL2']
method = np.median
n_spurious = None # use 0.7 quantile

df_chosen = filter_by_dict(df_total, dict_chosen)
df_calib = pd.DataFrame(index=[], 
                        columns=['frequencies', 'psd', 'psd_std', 
                                 'source', 'method'])

for source, df_source in df_chosen.groupby('source'):
    params = params_dict.get(source)
    wall_detector = WallDetector(params=params, 
                                 mic_type='audio_deck')
    
    with progressbar.ProgressBar(max_value=max(df_source.index)) as p:
        for j, row in df_source.iterrows():
            if row.appendix in chosen_experiments: 
                spec_masked, freqs_masked = wall_detector.fill_from_row(row, verbose=False)
            p.update(j)
            
    #wall_detector.remove_spurious_freqs(n_spurious, verbose=True)
    wall_detector.remove_spurious_freqs(n_spurious, verbose=True)
    psd, psd_std, freqs = wall_detector.get_frequency_slice_with_std(method=method)
    
    df_calib.loc[len(df_calib), :] = {
        'frequencies': freqs,
        'psd': psd, 
        'psd_std': psd_std, 
        'source':source, 
        'method':method
    }
    
    fig, ax  = plt.subplots()
    fig.set_size_inches(10, 5)
    fig.suptitle(f'source={source}')
    for i in range(psd.shape[0]):
        #ax.errorbar(freqs, psd[i], psd_std[i], label=f'mic{i}')
        ax.plot(freqs, psd[i], label=f'mic{i}')
    ax.set_yscale('log')
    plt.legend(loc='upper left')
    


In [ ]:
fname = f'results/calibration_results.pkl'
pd.to_pickle(df_calib, fname)
print('saved as', fname)
df_calib

In [ ]:
psd_all = np.concatenate(df_calib.psd.values, axis=1)
freqs_all = np.concatenate(df_calib.frequencies.values)
y_min = np.min(psd_all)
y_max = np.max(psd_all)
x_min = np.min(freqs_all) 
x_max = np.max(freqs_all) 

for i, row in df_calib.iterrows():
    
    fig, ax = plt.subplots()
    fig.set_size_inches(5, 5)
    title = f'{row.source}'
    ax.set_title(f'{title}, {row.method.__name__}')
    for i in range(row.psd.shape[0]):
        #ax.errorbar(row.frequencies, row.psd[i, :], row.psd_std[i, :])
        ax.plot(row.frequencies, row.psd[i, :], label=f'mic{i}')
    ax.set_yscale('log')
    ax.set_ylim(y_min, y_max)
    ax.set_xlim(x_min, x_max)
    ax.set_xlabel('frequency [Hz]')
    ax.set_ylabel('PSD')
    ax.legend(loc='lower right')
    
    if (row.method == 'median') and (row.source == 'sweep_buzzer'):
        fig.savefig(f'plots/calibration_{title}.pdf', bbox_inches='tight')

# 2. Chirp design

In [ ]:
from crazyflie_description_py.parameters import N_BUFFER, FS

# motors calibration
f_45000 = 671
f_55000 = 750
a = (f_55000 - f_45000) / (55000 - 45000)

thrusts = np.linspace(45000, 55000, 10)
#thrusts = np.linspace(50000, 60000, 10)

for DELTA in [50, 100, 200]:
    MIN_FREQ = 1000
    MAX_FREQ = 6000

    freqs = np.fft.rfftfreq(N_BUFFER, 1/FS)
    freqs = freqs[(freqs < MAX_FREQ) & (freqs > MIN_FREQ)]
    freqs_matrix = np.ones((len(thrusts), len(freqs)))

    for i, t in enumerate(thrusts):
        f = a * (t - 45000) + f_45000
        for k in range(30):
            freqs_matrix[i, (freqs < (k*f + DELTA)) & (freqs > (k*f - DELTA))] = 0

    mask = np.all(freqs_matrix > 0, axis=0)
    freqs_matrix[freqs_matrix.shape[0]//2, :] = 2*mask 
    fig, ax = plt.subplots()
    ax.pcolorfast(freqs, thrusts, freqs_matrix)
    ax.set_title(f'delta={DELTA}Hz')

    print(f'available for delta={DELTA}', freqs[mask])

In [ ]:
from audio_bringup.helpers import get_filename
from evaluate_data import read_df_from_wav, read_df
from frequency_analysis import get_spectrogram
from crazyflie_description_py.parameters import N_BUFFER, FS

chosen_freqs_approx = np.array([3013, 3120, 3530, 3680, 4150, 4270, 5130, 5212, 6430, 6690, 8000, 8420, 10940, 13140, 14920]) # 4270, 

frequencies_embed = np.fft.rfftfreq(N_BUFFER, 1/FS)
chosen_bins = [np.argmin(np.abs(f - frequencies_embed)) for f in chosen_freqs_approx]
chosen_freqs = frequencies_embed[chosen_bins]

# harmonics

higher_approx = chosen_freqs_approx * 3
chosen_bins_higher = [np.argmin(np.abs(f - frequencies_embed)) for f in higher_approx if f < 16000]
chosen_freqs_higher = frequencies_embed[chosen_bins_higher]

lower_approx = list(chosen_freqs_approx[[2, 3, 4]] / 3) #, 11, 12, 13, 14]] / 3)
chosen_bins_lower = [np.argmin(np.abs(f - frequencies_embed)) for f in lower_approx]
chosen_freqs_lower = frequencies_embed[chosen_bins_lower]

buzzer_freqs = sorted(list(chosen_freqs) + list(chosen_freqs_lower))
print('buzzer freqs', len(buzzer_freqs), buzzer_freqs)

all_freqs = sorted(list(chosen_freqs) + list(chosen_freqs_higher) + list(chosen_freqs_lower) 
                   + list(chosen_freqs_lower * 5) + list(chosen_freqs_lower * 7))
print('all freqs', len(all_freqs), all_freqs)

In [ ]:
frequencies_embed = np.fft.rfftfreq(N_BUFFER, 1/FS)
all_bins = [np.argmin(np.abs(f - frequencies_embed)) for f in all_freqs]
print('bins:', all_bins)

out_dir = "../crazyflie-audio/firmware/audio_shield_firmware/Core/Inc"
out_name = "sweep_hard_bins.h"
fname = f"{out_dir}/{out_name}"

with open(fname, "w+") as f:
    f.write(f"#ifndef __SWEEP_HARD_BINS_H \n#define __SWEEP_HARD_BINS_H\n\n")

    f.write(f"uint16_t sweep_hard_bins[32] = " + r"{")
    [f.write(f"{s:.0f}, ",) for s in all_bins[:-1]]
    [f.write(f"{s:.0f}",) for s in all_bins[-1:]]
    f.write(r"};" + " \n\n")

    f.write(f"#endif /* __SWEEP_HARD_BINS_H */")
print('wrote as', fname)

# motors at hover

## measurement mics

In [ ]:
#kwargs = dict(
#    degree=0,
#    props=False,
#    snr=True,
#    motors=True,
#    source=None,
#    distance=0
#)
#exp_name = '2020_11_30_wall_hover'; 

#kwargs = dict(
#    degree=0,
#    props=0,
#    snr=2,
#    motors=True,
#    source=None,
#    distance=51
#)
#exp_name = '2020_12_18_stepper'; 

kwargs = dict(
    degree=0,
    props=False,
    snr=2,
    motors=True,
    source=None,
    distance=None
)
exp_name = '2020_12_18_flying'; 

fname_motors = get_filename(**kwargs)
df_motors = read_df_from_wav(f'../experiments/{exp_name}/export/{fname_motors}.wav', n_buffer=N_BUFFER)
df_motors.loc[:, 'seconds'] = (df_motors.timestamp.values - df_motors.iloc[0].timestamp) / 1000.0

In [ ]:
frequencies_slices = [1750, 2375, 3125, 3875]

spec_motors_all, frequencies_motors = get_spectrogram(df_motors)
spec_motors = np.mean(spec_motors_all, axis=-2)

#times_motors = np.arange(spec_motors.shape[1]) #
times_motors = df_motors.seconds.values
mask_times = (times_motors < 20) & (times_motors > 7)

avg_motor_psd = np.mean(spec_motors[:, mask_times]**2, axis=1) # time range chosen from plot
min_freq_plot = 0 #2500
for max_freq_plot in [4000, 16000]:
    mask_freq = (frequencies_motors < max_freq_plot) & (frequencies_motors > min_freq_plot)
    
    spec = spec_motors[mask_freq, :]
    #spec = spec[:, mask_times]
    
    fig, ax = plt.subplots()
    fig.set_size_inches(10, 5)
    ax.pcolorfast(times_motors, frequencies_motors[mask_freq], np.log(spec))
    ax.axvline(times_motors[mask_times][0], color='white')
    ax.axvline(times_motors[mask_times][-1], color='white')
    [ax.axhline(f, color='red', ls=':') for f in frequencies_slices]
    ax.set_xlabel('time [s]')
    ax.set_ylabel('frequency [Hz]')
    fname = f'plots/{exp_name}_spec_zoom{max_freq_plot}.pdf'
    fig.savefig(fname, bbox_inches='tight')
    print('saved as', fname)
    
    fig = plt.figure()
    plt.semilogy(frequencies_motors[mask_freq], avg_motor_psd[mask_freq])
    [plt.axvline(f, color='red', ls=':') for f in frequencies_slices]
    plt.xlabel('frequency [Hz]')
    plt.grid(which='both')
    plt.ylabel('PSD')
    
    fname = f'plots/{exp_name}_avg_zoom{max_freq_plot}.pdf'
    fig.savefig(fname, bbox_inches='tight')
    print('saved as', fname)
pass

##  drone mics

In [ ]:
df, df_pos = read_df(**kwargs, exp_name=exp_name)

In [ ]:
from evaluate_data import read_df_others
df_status, df_commands = read_df_others(**kwargs, exp_name=exp_name)

plt.figure()
plt.scatter(df.timestamp.values, [0]*len(df), color='C0')
plt.scatter(df_pos.timestamp.values, [3]*len(df_pos), color='C0')
plt.scatter(df_status.timestamp.values, [1]*len(df_status), color='C0')
plt.scatter(df_commands.timestamp.values, [2]*len(df_commands), color='C0')

min_time = min([
    df.iloc[0].timestamp, 
    df_pos.iloc[0].timestamp, 
    df_status.iloc[0].timestamp, 
    df_commands.iloc[0].timestamp])

times_commands = (df_commands.timestamp.values - min_time) / 1000
times_status = (df_status.timestamp - min_time) / 1000
times_pos = (df_pos.timestamp - min_time) / 1000

motors_pwm = np.array([*df_commands.loc[:, "motors_pwm"]])
max_pwm = 2**16-1
fig, ax = plt.subplots()
fig.set_size_inches(10, 5)
for i in range(motors_pwm.shape[1]):
    ax.scatter(times_commands, motors_pwm[:, i], label=f"motor {i}")
ax.set_ylim(45000, 62000)
ax.legend(loc='upper right')
ax.set_ylabel(f'thrust command (out of {max_pwm})')
ax.set_xlabel('time [s]')
fname = 'plots/commands_hover.pdf'
fig.savefig(fname, boox_inches='tight')
print('saved as', fname)

fig, axs = plt.subplots(3, sharex=True)
fig.set_size_inches(15, 10)
axs[0].plot(times_status, df_status.vbat, label="vbat")
axs[1].plot(times_pos, df_pos.dx, label="dx")
axs[1].plot(times_pos, df_pos.dy, label="dy")
axs[1].set_ylim(-10, 10)

axs[2].plot(times_pos, df_pos.yaw_rate_deg, label="yaw_rate_deg")
[ax.legend(loc='lower right') for ax in axs]

# buzzer chirp

In [ ]:
kwargs = dict(
    degree=0,
    props=False,
    snr=False,
    motors=False,
    source='sweep_all',
    distance=50
)
exp_name = '2020_12_2_chirp'; 
n_buffer = N_BUFFER 
fname_chirp = get_filename(**kwargs)
fname = f'results/{exp_name}_real.pkl'
try:
    df_chirp = pd.read_pickle(fname)
    print('read', fname)
except:
    print('run wall_analysis to parse')
df_chirp.tail()

In [ ]:
#from frequency_analysis import extract_linear_psd
from frequency_analysis import get_spectrogram_raw

row = df_chirp[df_chirp.mic_type=='measurement'].iloc[0]
spec_chirp_all, frequencies_chirp = get_spectrogram_raw(row.frequencies_matrix, row.stft)

plot_spectrogram(row.seconds, frequencies_chirp, spec_chirp_all)

In [ ]:
from frequency_analysis import apply_linear_mask, apply_box_mask
from frequency_analysis import psd_df_from_spec, get_index_matrix
from frequency_analysis import extract_psd 

delta = 50
slope = 16000 / 500 #21170 
offset = 0
spec_masked_all, freqs_masked = apply_linear_mask(spec_chirp_all, 
                                                  frequencies_chirp, 
                                                  times=row.seconds, 
                                                  offset=offset, 
                                                  slope=slope, 
                                                  delta=delta)
spec_masked_all, freqs_masked = apply_box_mask(spec_masked_all, 
                                               freqs_masked, 
                                               times=row.seconds, 
                                               min_time=60)

fig, ax = plt.subplots()
fig.set_size_inches(10, 10)
plot_spectrogram(x=row.seconds, y=freqs_masked, 
                 matrix=spec_masked_all, ax=ax)

index_matrix = get_index_matrix(spec_masked_all)
ax.plot(row.seconds, freqs_masked[index_matrix[0, :]], color='white', ls=':')

psd_df = psd_df_from_spec(spec_masked_all, freqs_masked, index_matrix)
psd, freq_psd, psd_std = extract_psd(psd_df)

In [ ]:
fig = plt.figure()
fig.set_size_inches(7, 7)
plt.loglog(freq_psd, psd[0], marker='o')
plt.title('amplitude of buzzer frequency sweep')
plt.grid(which='both')
plt.xlabel('frequency [Hz]')

# motors at constant thrust and old frequency sweep

In [ ]:
kwargs = dict(
    degree=0,
    props=False,
    snr=False,
)
exp_name = '2020_10_14_static_new'; 
fname_motors = get_filename(motors=True, source=None, distance=None, **kwargs)

df_motors = read_df_from_wav(f'../experiments/{exp_name}/export/{fname_motors}.wav')

exp_name = '2020_11_26_wall'; 
fname_nomotors = get_filename(motors=False, source='sweep', distance=49, **kwargs)
df_nomotors = read_df_from_wav(f'../experiments/{exp_name}/export/{fname_nomotors}.wav')

In [ ]:
from bin_selection import select_frequencies
#sys.path.append(f'../experiments/{exp_name}/')
#import params as params_buzz
from crazyflie_description_py.parameters import SOUND_EFFECTS, N_BUFFER, FS

max_time = 65
max_freq_plot = 15000

#__, (min_freq, max_freq), *_ = SOUND_EFFECTS['sweep']
min_freq = 1000; max_freq = 9000
bins = select_frequencies(min_freq=min_freq, max_freq=max_freq, n_buffer=N_BUFFER, fs=FS)
frequencies_embed = np.fft.rfftfreq(N_BUFFER, 1/FS)
print(frequencies_embed[bins])

for df in [df_motors, df_nomotors]:
    fig = plt.figure()
    fig.set_size_inches(15, 10)
    times = df.timestamp.values / 1000

    spec_all, frequencies = get_spectrogram(df)
    spec = np.mean(spec_all, axis=-2)
    spec = spec[frequencies < max_freq_plot, :]
    spec = spec[:, times < max_time]
    plt.pcolormesh(times[times < max_time], frequencies[frequencies < max_freq_plot], 
                   np.log(spec))
    [plt.axhline(frequencies_embed[b], color='white', ls=':') for b in bins]
    plt.xlabel('time [s]')
    plt.ylabel('frequency [Hz]')

In [ ]:
from evaluate_data import read_df_from_wav
df_wav = read_df_from_wav('../experiments/2021_01_07_snr_study/export/motors_nosnr_noprops_mono1750.wav')
spec_all, freqs = get_spectrogram(df_wav)
spec = np.mean(spec_all, axis=-2)

In [ ]:
freqs = df_wav.iloc[0].frequencies
fig, ax = plt.subplots()
ax.pcolorfast(range(spec.shape[0]), freqs, np.log10(spec))